# The objective of this tutorial is to use the AppEEARS API to extract LST for two wildfires in Montana

#### Select code adapted from: https://git.earthdata.nasa.gov/projects/LPDUR/repos/appeears-api-getting-started/browse

### To extract wildfire polygons from Google Earth Engine, click the link below
### https://code.earthengine.google.com/cc8a5e7886d2a02719f87ce13bc7dc40

In [1]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json # getpass needed for Earthdata login
import geopandas as gpd

In [2]:
# Set input directory, change working directory
inDir = '/home/marie/wildfire/LST'           # working directory
os.chdir(inDir)                                      # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

In [3]:
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username: ········
Enter NASA Earthdata Login Password: ········


#### The requests package will post your username and password. The token received will be used later to submit your request.

In [4]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'x0ahhDEY3SPOPLMeCna-P5HQGTpjKldmRJvRMRswmNwmLuhd8jmxkOtf5quPaoGnVKqwaRzAVs27r7_tfzLnoQ',
 'expiration': '2022-10-20T01:29:40Z'}

#### Available products from appears
https://appeears.earthdatacloud.nasa.gov/products

In [5]:
# The get() method returns the value of the item with the specified key.
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service

In [6]:
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
products['ECO2LSTE.001']                                         # Print information for the product of interest ECO2LSTE.001 LST Product

{'Product': 'ECO2LSTE',
 'Platform': 'ECOSTRESS',
 'Description': 'Land Surface Temperature & Emissivity (LST&E)',
 'RasterType': 'Swath',
 'Resolution': '70m',
 'TemporalGranularity': 'ISS-dependent',
 'Version': '001',
 'Available': True,
 'DocLink': 'https://doi.org/10.5067/ECOSTRESS/ECO2LSTE.001',
 'Source': 'LP DAAC',
 'TemporalExtentStart': '2018-07-09',
 'TemporalExtentEnd': 'Present',
 'Deleted': False,
 'DOI': '10.5067/ECOSTRESS/ECO2LSTE.001',
 'ProductAndVersion': 'ECO2LSTE.001'}

#### I would like explore what layers are within the LST product

In [7]:
prods = ['ECO2LSTE.001']     # Start a list for products to be requested, beginning with ECO2LSTE.001
lst_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the only product (index 0) in the list: ECO2LSTE.001
list(lst_response.keys())

['SDS_Emis1',
 'SDS_Emis1_err',
 'SDS_Emis2',
 'SDS_Emis2_err',
 'SDS_Emis3',
 'SDS_Emis3_err',
 'SDS_Emis4',
 'SDS_Emis4_err',
 'SDS_Emis5',
 'SDS_Emis5_err',
 'SDS_EmisWB',
 'SDS_LST',
 'SDS_LST_err',
 'SDS_PWV',
 'SDS_QC']

#### I'm only interested in SDS_LST right now, so I will use that dictionary key to learn more about that layer from the lst_response variable

In [8]:
lst_response['SDS_LST']

{'AddOffset': 0.0,
 'Available': True,
 'DataType': 'uint16',
 'Description': 'Land Surface Temperature',
 'Dimensions': ['FakeDim0', 'FakeDim1'],
 'FillValue': 0.0,
 'Group': '',
 'IsQA': False,
 'Layer': 'SDS_LST',
 'OrigDataType': 'uint16',
 'OrigValidMax': 65535,
 'OrigValidMin': 7500,
 'QualityLayers': "['SDS_QC']",
 'QualityProductAndVersion': 'ECO2LSTE.001',
 'ScaleFactor': 0.02,
 'Units': 'K',
 'ValidMax': 1310.7,
 'ValidMin': 150.0,
 'XSize': 5400,
 'YSize': 5632}

In [9]:
# Adding QC layer
layers = [(prods[0],'SDS_LST'),(prods[0],'SDS_QC')]  # Create tupled list linking desired product with desired layers

#### Here we are created a dictionary list from our tupled list (the layers) that stores LST and LST QC. These will be used to insert into the json file when we submit our requests.

In [10]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'},
 {'layer': 'SDS_QC', 'product': 'ECO2LSTE.001'}]

# Submit Request

### Document this part

In [11]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

In [12]:
czHorse = gpd.read_file('crazyHorseShapefile.shp'.format(inDir + os.sep + 'Data' + os.sep)) # Read in shapefile as dataframe using geopandas
print(czHorse.head())                                                # Print first few lines of dataframe

  Comment Incid_Type  High_T  Mod_T  NoData_T BurnBndLon  Low_T  \
0    None   Wildfire     660    371      -970   -113.797    120   

         Ig_Date Map_Prog  dNBR_stdDv  ...               Event_ID  Perim_ID  \
0  1060498800000     MTBS       -9999  ...  MT4743511379720030810      None   

  BurnBndLat dNBR_offst BurnBndAc IncGreen_T  irwinID          Post_ID  \
0     47.435         15     11423       -150     None  504102720040925   

   Asmnt_Type                                           geometry  
0    Extended  POLYGON ((-113.86393 47.43105, -113.86337 47.4...  

[1 rows x 23 columns]


### Set projection of choice LEFT OFF HERE

In [14]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
# projections                                               # If you would like you can print projections and information

In [15]:
# Figure out how to insert the geographic projection here
projs = {}                                  # Create a dictionary with one entry, we're going to set the projection to geographic
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north']

### Convert to json

In [16]:
czHorseJ = czHorse.to_json() 
czHorseJ = json.loads(czHorseJ)                                            # Convert to json format

### Compile JSON

In [17]:
task_name = input('Enter a Task Name: ') # We will use 'Crazy_Horse_Fire' as the task name, this will be the name of the folder when downloaded, better to not include spaces!

Enter a Task Name: Crazy Horse Fire


In [18]:
task_type = ['area']        # Our task is an area, but you could also select point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff']             # Set output file format type
startDate = '07-01-2020'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '07-31-2020'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = True                   # Specify True for a recurring date range
yearRange = [2020,2021]            # if recurring = True, set yearRange, change start/end date to MM-DD

In [21]:
task = {
    'task_type': task_type[0],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': czHorseJ,
    }
}

## Submit a task

In [22]:
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response                                                                 # Print task response

{'task_id': 'd8f5ba50-ebaf-4577-b896-319fefe00194', 'status': 'pending'}

In [29]:
task_id = task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
status_response                                                        

{'error': None,
 'params': {'geo': {'type': 'FeatureCollection',
   'features': [{'id': '0',
     'type': 'Feature',
     'geometry': {'type': 'Polygon',
      'coordinates': [[[-113.86393470121736, 47.43105210536035],
        [-113.86336830745545, 47.431957302996146],
        [-113.86190576526579, 47.43285803738252],
        [-113.86319448129103, 47.435043015828356],
        [-113.861459866494, 47.43621133275554],
        [-113.85910101708157, 47.435105439847526],
        [-113.85832955184567, 47.433558119538134],
        [-113.85596627389876, 47.43245672240649],
        [-113.85473995035754, 47.43049473741957],
        [-113.85475342733673, 47.429317544488576],
        [-113.85398644311284, 47.42777022770427],
        [-113.85094533555925, 47.42793073551312],
        [-113.84732448455114, 47.427226205004175],
        [-113.84434583248164, 47.42761419626169],
        [-113.84083654065941, 47.42854612287181],
        [-113.83796486489392, 47.430566072127434],
        [-113.836422026346

## Retrieve task status

In [31]:
params = {'limit': 2, 'pretty': True} # Limit API response to 2 most recent entries, return as pretty json

In [32]:
tasks_response = r.get('{}task'.format(api), params=params, headers=head).json() # Query task service, setting params and header 
tasks_response   

[{'error': None,
  'params': {'dates': [{'endDate': '07-31-2020', 'startDate': '07-01-2020'}],
   'layers': [{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'},
    {'layer': 'SDS_QC', 'product': 'ECO2LSTE.001'}],
   'output': {'format': {'type': 'geotiff'}, 'projection': 'geographic'}},
  'status': 'done',
  'created': '2022-10-19T22:22:11.238879',
  'task_id': 'd8f5ba50-ebaf-4577-b896-319fefe00194',
  'updated': '2022-10-19T22:35:46.857796',
  'user_id': 'marie3.johnson@umontana.edu',
  'attempts': 1,
  'estimate': {'request_size': 15087918359.146465},
  'retry_at': None,
  'completed': '2022-10-19T22:35:46.834877',
  'has_swath': True,
  'task_name': 'Crazy Horse Fire',
  'task_type': 'area',
  'api_version': 'v1',
  'svc_version': '3.14',
  'web_version': None,
  'size_category': '0',
  'has_nsidc_daac': False,
  'expires_on': '2022-11-18T22:35:46.857796'},
 {'error': None,
  'params': {'dates': [{'endDate': '12-31-2020',
     'recurring': False,
     'startDate': '01-01-2019',
     '

In [33]:
# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

done


## Download Request

In [34]:
# destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
practDir = '/home/marie/wildfire/LST'
destDir = os.path.join(practDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

In [35]:
bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
bundle                                                    # Print bundle contents

{'files': [{'sha256': '30ac88a235f0ddbf4cea83fcb34b6d022b8ec9bb09992c50a09a3accfb2e9e47',
   'file_id': '6034c2e7-d3ca-426b-a752-827faecd3f21',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020186135802_aid0001.tif',
   'file_size': 22875,
   'file_type': 'tif'},
  {'sha256': '9dcbecf06e1aa48330f89855a47b9bdf0c335345bb96fb8b12742b3d2d207224',
   'file_id': '6c999cc0-ef37-4892-8029-968af43a38dd',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020187095615_aid0001.tif',
   'file_size': 21742,
   'file_type': 'tif'},
  {'sha256': 'a66a75223d3c9e022eedf77b0bc0627cc84bcf61d038c0c3222c67a68d27bddd',
   'file_id': 'a249650b-6dc8-4d13-b6b9-0c5cc69a86b8',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020188090905_aid0001.tif',
   'file_size': 21614,
   'file_type': 'tif'},
  {'sha256': '86fda79c58472ca0d02522bdc651f7bc87662ddf28242ebe5932a67e78a7ebdf',
   'file_id': 'f13209ab-c5f2-4e97-80bc-cb9012df3227',
   'fil

In [37]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files    

{'6034c2e7-d3ca-426b-a752-827faecd3f21': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020186135802_aid0001.tif',
 '6c999cc0-ef37-4892-8029-968af43a38dd': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020187095615_aid0001.tif',
 'a249650b-6dc8-4d13-b6b9-0c5cc69a86b8': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020188090905_aid0001.tif',
 'f13209ab-c5f2-4e97-80bc-cb9012df3227': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020190122451_aid0001.tif',
 'b517293f-7141-4f31-af44-77b1cfe983fe': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020194105045_aid0001.tif',
 '8069a02e-1a39-4d37-b3ca-cb0a86ac0ada': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020195064821_aid0001.tif',
 '70dfb04c-2221-4c21-90a0-4ffbb7a57925': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020196060053_aid0001.tif',
 'e3ad2d2b-f743-49c8-b1dc-4db11843ee54': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020198091544_aid000

In [38]:
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
    if files[f].endswith('.tif'):
        filename = files[f].split('/')[1]
    else:
        filename = files[f] 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))

Downloaded files can be found at: /home/marie/wildfire/LST/Crazy Horse Fire


## Submit another task for the Little Salmon Fire in the Bob Marshall Wilderness

In [52]:
ltSalm = gpd.read_file('littleSalmonShapefile.shp'.format(inDir + os.sep + 'Data' + os.sep)) # Read in shapefile as dataframe using geopandas
# print(ltSalm.head())                                                # Print first few lines of dataframe

### Convert to JSON

In [53]:
ltSalmJ = ltSalm.to_json() 
ltSalmJ = json.loads(ltSalmJ)                                            # Convert to json format

### Compile JSON

In [54]:
task_name = input('Enter a Task Name: ') # We will use 'Little_Salmon_Fire' as the task name, this will be the name of the folder when downloaded, better to not include spaces!

Enter a Task Name: Little_Salmon_Fire


#### Update task formatting and date

In [55]:
task_type = ['area']        # Our task is an area, but you could also select point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff']             # Set output file format type
startDate = '07-01-2020'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '07-31-2020'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = True                   # Specify True for a recurring date range
yearRange = [2020,2021]            # if recurring = True, set yearRange, change start/end date to MM-DD

In [56]:
task = {
    'task_type': task_type[0],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': ltSalmJ,
    }
}

## Submit the second task

In [57]:
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response                                                                 # Print task response

{'task_id': '0df7edde-a4f1-4938-84a8-92163f53044d', 'status': 'pending'}

### Check the new task status

In [59]:
task_response                                                                 # Print task response

{'task_id': '0df7edde-a4f1-4938-84a8-92163f53044d', 'status': 'pending'}

In [60]:
bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
bundle                                                    # Print bundle contents

{'files': [{'sha256': '30ac88a235f0ddbf4cea83fcb34b6d022b8ec9bb09992c50a09a3accfb2e9e47',
   'file_id': '6034c2e7-d3ca-426b-a752-827faecd3f21',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020186135802_aid0001.tif',
   'file_size': 22875,
   'file_type': 'tif'},
  {'sha256': '9dcbecf06e1aa48330f89855a47b9bdf0c335345bb96fb8b12742b3d2d207224',
   'file_id': '6c999cc0-ef37-4892-8029-968af43a38dd',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020187095615_aid0001.tif',
   'file_size': 21742,
   'file_type': 'tif'},
  {'sha256': 'a66a75223d3c9e022eedf77b0bc0627cc84bcf61d038c0c3222c67a68d27bddd',
   'file_id': 'a249650b-6dc8-4d13-b6b9-0c5cc69a86b8',
   'file_name': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020188090905_aid0001.tif',
   'file_size': 21614,
   'file_type': 'tif'},
  {'sha256': '86fda79c58472ca0d02522bdc651f7bc87662ddf28242ebe5932a67e78a7ebdf',
   'file_id': 'f13209ab-c5f2-4e97-80bc-cb9012df3227',
   'fil

In [61]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files    

{'6034c2e7-d3ca-426b-a752-827faecd3f21': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020186135802_aid0001.tif',
 '6c999cc0-ef37-4892-8029-968af43a38dd': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020187095615_aid0001.tif',
 'a249650b-6dc8-4d13-b6b9-0c5cc69a86b8': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020188090905_aid0001.tif',
 'f13209ab-c5f2-4e97-80bc-cb9012df3227': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020190122451_aid0001.tif',
 'b517293f-7141-4f31-af44-77b1cfe983fe': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020194105045_aid0001.tif',
 '8069a02e-1a39-4d37-b3ca-cb0a86ac0ada': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020195064821_aid0001.tif',
 '70dfb04c-2221-4c21-90a0-4ffbb7a57925': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020196060053_aid0001.tif',
 'e3ad2d2b-f743-49c8-b1dc-4db11843ee54': 'ECO2LSTE.001_2020183_to_2020213/ECO2LSTE.001_SDS_LST_doy2020198091544_aid000

In [63]:
practDir = '/home/marie/wildfire/LST'
# The task_name within destDir has to be updated
destDir = os.path.join(practDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

In [64]:
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = 'True')                                # Get a stream to the bundle file
    if files[f].endswith('.tif'):
        filename = files[f].split('/')[1]
    else:
        filename = files[f] 
    filepath = os.path.join(destDir, filename)                                           # Create output file path
    with open(filepath, 'wb') as f:                                                       # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))

Downloaded files can be found at: /home/marie/wildfire/LST/Little_Salmon_Fire
